In [1]:
# Importer les modules
from keras.models import Sequential #Initialisation d'un ANN
from keras.layers import Conv2D     #Etape Convolution des images 2D
from keras.layers import MaxPooling2D #Etape MaxPoling
from keras.layers import Flatten   #Etape Flating 
from keras.layers import Dense     # Ajouter des couches complétement connectées

#Partie1 construire notre CNN

Using TensorFlow backend.


In [2]:
#Initialiser notre CNN
classifier = Sequential()

# Etape1 appliquer la convolution pour avoir la couche convolution
#32=nb de features detectors=puiss de 2
#dim de features=3*3
#input_shape=on le specifie à la première couche de concolution 64*64 (toutes les images seront redimensionnées à cette taille) et 3 indique que les images sont en couleurs
#relu=fonction d'activation redresseur 
classifier.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))

# Etape2 on applique le Pooling
 #taille de matrice de pooling size =2*2
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Ajouter une deuxième couche de convolution pour améliorer la précision du modèle de départ
classifier.add(Conv2D(32, (3, 3), activation = 'relu'))
classifier.add(MaxPooling2D(pool_size = (2, 2)))

# Etape3 on applique le Flattening
classifier.add(Flatten())

# Etape 4 on ajoute couche cachée complétement connecté
classifier.add(Dense(units = 128, activation = 'relu'))
classifier.add(Dense(units = 1, activation = 'sigmoid'))

#compiler notre CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

# Partie 2 : augmenter notre jeu de données et entrainer notre CNN

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, # l'echelle de chaque pixel [0-1]
                                   shear_range = 0.2, #transvection des images
                                   zoom_range = 0.2, # zoomer des images
                                   horizontal_flip = True) # rotation

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (64, 64),
                                                 batch_size = 32, #lot d'observationspour les maj
                                                 class_mode = 'binary')

test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (64, 64),
                                            batch_size = 32,
                                            class_mode = 'binary')

classifier.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5

import numpy as np
from keras.preprocessing import image

#faire une nouvelle prédicition sur de nouveaux images
test_image = image.load_img('dataset/single_prediction/cat_or_dog_2.jpg', target_size = (64, 64))#transformer l'image à la taille 64*64
test_image = image.img_to_array(test_image) #transformer en image en 3 dimensions 64,64,3
test_image = np.expand_dims(test_image, axis = 0)#ajouter une dimension au début axis=0 1,64,64,3
result = classifier.predict(test_image)



training_set.class_indices
if result[0][0] == 1:
    prediction = 'dog'
else:
    prediction = 'cat'
    
print(prediction)   

#précision sur le test 81% et sur train 91% problème de sur-enptrainement

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 244s 974ms/step - loss: 0.6729 - accuracy: 0.5779 - val_loss: 0.3946 - val_accuracy: 0.6810
Epoch 2/25
250/250 [==============================] - 80s 319ms/step - loss: 0.5994 - accuracy: 0.6793 - val_loss: 0.4348 - val_accuracy: 0.6935
Epoch 3/25
250/250 [==============================] - 83s 332ms/step - loss: 0.5576 - accuracy: 0.7115 - val_loss: 0.4989 - val_accuracy: 0.7495
Epoch 4/25
250/250 [==============================] - 86s 342ms/step - loss: 0.5267 - accuracy: 0.7324 - val_loss: 0.7200 - val_accuracy: 0.7545
Epoch 5/25
250/250 [==============================] - 84s 336ms/step - loss: 0.4924 - accuracy: 0.7584 - val_loss: 0.4895 - val_accuracy: 0.7455
Epoch 6/25
250/250 [==============================] - 85s 339ms/step - loss: 0.4846 - accuracy: 0.7635 - val_loss: 0.4488 - val_accuracy: 0.7850
Epoch 7/25
250/250 [=========================

In [3]:
#améliorer le modèle en ajoutantl'algo SGD (stochastic gradient descent)
from keras.optimizers import SGD

model_SGD = Sequential()
model_SGD.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model_SGD.add(MaxPooling2D(pool_size = (2, 2)))
model_SGD.add(Conv2D(32, (3, 3), activation = 'relu'))
model_SGD.add(MaxPooling2D(pool_size = (2, 2)))
model_SGD.add(Flatten())
model_SGD.add(Dense(units = 128, activation = 'relu'))
model_SGD.add(Dense(units = 1, activation = 'sigmoid'))

# Compile model
sgd = SGD(lr=0.01, momentum=0.8)
model_SGD.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_SGD.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5
#train précision 84% test 76%

Epoch 1/25
250/250 [==============================] - 85s 339ms/step - loss: 0.6871 - accuracy: 0.5520 - val_loss: 0.7065 - val_accuracy: 0.6105
Epoch 2/25
250/250 [==============================] - 81s 326ms/step - loss: 0.6705 - accuracy: 0.5886 - val_loss: 0.7381 - val_accuracy: 0.5635
Epoch 3/25
250/250 [==============================] - 84s 335ms/step - loss: 0.6479 - accuracy: 0.6319 - val_loss: 0.5564 - val_accuracy: 0.6455
Epoch 4/25
250/250 [==============================] - 86s 342ms/step - loss: 0.6260 - accuracy: 0.6510 - val_loss: 0.6948 - val_accuracy: 0.6785
Epoch 5/25
250/250 [==============================] - 86s 343ms/step - loss: 0.6023 - accuracy: 0.6734 - val_loss: 0.7116 - val_accuracy: 0.6675
Epoch 6/25
250/250 [==============================] - 86s 343ms/step - loss: 0.5815 - accuracy: 0.6964 - val_loss: 0.5568 - val_accuracy: 0.7125
Epoch 7/25
250/250 [==============================] - 87s 347ms/step - loss: 0.5626 - accuracy: 0.7056 - val_loss: 0.6292 - val_ac

In [6]:
#améliorer le modèle en utilisant le dropout et l'algo SGD
#dropout is only used after the pooling layers
from keras.layers import Dropout


model_SGD_dropout = Sequential()
model_SGD_dropout.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))
model_SGD_dropout.add(Dropout(0.5))

model_SGD_dropout.add(Conv2D(32, (3, 3), activation = 'relu'))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))
model_SGD_dropout.add(Dropout(0.5))

model_SGD_dropout.add(Flatten())
model_SGD_dropout.add(Dense(units = 128, activation = 'relu'))
model_SGD_dropout.add(Dense(units = 1, activation = 'sigmoid'))

# Compile model
sgd = SGD(lr=0.01, momentum=0.8)
model_SGD_dropout.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_SGD_dropout.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5

#train précision 75% test précision 75%

Epoch 1/25
250/250 [==============================] - 85s 339ms/step - loss: 0.6930 - accuracy: 0.5211 - val_loss: 0.6925 - val_accuracy: 0.5690
Epoch 2/25
250/250 [==============================] - 86s 345ms/step - loss: 0.6821 - accuracy: 0.5619 - val_loss: 0.6894 - val_accuracy: 0.5195
Epoch 3/25
250/250 [==============================] - 86s 344ms/step - loss: 0.6723 - accuracy: 0.5835 - val_loss: 0.6937 - val_accuracy: 0.5740
Epoch 4/25
250/250 [==============================] - 94s 375ms/step - loss: 0.6600 - accuracy: 0.6121 - val_loss: 0.5870 - val_accuracy: 0.6545
Epoch 5/25
250/250 [==============================] - 98s 392ms/step - loss: 0.6421 - accuracy: 0.6432 - val_loss: 0.5851 - val_accuracy: 0.6505
Epoch 6/25
250/250 [==============================] - 92s 369ms/step - loss: 0.6285 - accuracy: 0.6566 - val_loss: 0.6606 - val_accuracy: 0.6500
Epoch 7/25
250/250 [==============================] - 95s 381ms/step - loss: 0.6203 - accuracy: 0.6543 - val_loss: 0.5085 - val_ac

In [8]:
#améliorer le modèle en utilisant le dropout spatial
#dropout with convolutional neural networks is to dropout entire feature maps from the 
#convolutional layer which are then not used during pooling
#spatial dropout for a CNN
from keras.layers import SpatialDropout2D
model_SGD_dropout = Sequential()

model_SGD_dropout.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model_SGD_dropout.add(SpatialDropout2D(0.5))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))

model_SGD_dropout.add(Conv2D(32, (3, 3), activation = 'relu'))
model_SGD_dropout.add(SpatialDropout2D(0.5))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))


model_SGD_dropout.add(Flatten())
model_SGD_dropout.add(Dense(units = 128, activation = 'relu'))
model_SGD_dropout.add(Dense(units = 1, activation = 'sigmoid'))

# Compile model
sgd = SGD(lr=0.01, momentum=0.8)
model_SGD_dropout.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])
model_SGD_dropout.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5

#train précision=70% testprécision=74%

Epoch 1/25
250/250 [==============================] - 94s 375ms/step - loss: 0.6929 - accuracy: 0.5280 - val_loss: 0.6596 - val_accuracy: 0.5100
Epoch 2/25
250/250 [==============================] - 105s 419ms/step - loss: 0.6868 - accuracy: 0.5554 - val_loss: 0.6841 - val_accuracy: 0.6350
Epoch 3/25
250/250 [==============================] - 109s 438ms/step - loss: 0.6812 - accuracy: 0.5658 - val_loss: 0.6688 - val_accuracy: 0.6115
Epoch 4/25
250/250 [==============================] - 102s 409ms/step - loss: 0.6703 - accuracy: 0.5905 - val_loss: 0.6462 - val_accuracy: 0.6310
Epoch 5/25
250/250 [==============================] - 94s 378ms/step - loss: 0.6619 - accuracy: 0.6035 - val_loss: 0.6460 - val_accuracy: 0.6575
Epoch 6/25
250/250 [==============================] - 92s 367ms/step - loss: 0.6553 - accuracy: 0.6114 - val_loss: 0.5445 - val_accuracy: 0.6735
Epoch 7/25
250/250 [==============================] - 92s 366ms/step - loss: 0.6485 - accuracy: 0.6214 - val_loss: 0.5786 - val

C:\Users\SABRINA NEMEUR\Documents\Anaconda3\envs\deeplearningaz\lib\site-packages\keras\utils\data_utils.py:616: UserWarning: The input 127 could not be retrieved. It could be because a worker has died.
  UserWarning)


250/250 [==============================] - 621s 2s/step - loss: 0.5878 - accuracy: 0.6854 - val_loss: 0.4287 - val_accuracy: 0.7070
Epoch 22/25
250/250 [==============================] - 141s 563ms/step - loss: 0.5832 - accuracy: 0.6927 - val_loss: 0.4339 - val_accuracy: 0.7205
Epoch 23/25
250/250 [==============================] - 82s 328ms/step - loss: 0.5825 - accuracy: 0.6871 - val_loss: 0.5823 - val_accuracy: 0.7300
Epoch 24/25
250/250 [==============================] - 87s 350ms/step - loss: 0.5802 - accuracy: 0.6910 - val_loss: 0.4816 - val_accuracy: 0.7345
Epoch 25/25
250/250 [==============================] - 89s 355ms/step - loss: 0.5734 - accuracy: 0.6970 - val_loss: 0.4779 - val_accuracy: 0.7410


In [11]:
from keras.constraints import maxnorm
#améliorer le modèle en utilisant le dropout sur au début du ANN et on ajoutant une couche complétement connectée
model_SGD_dropout = Sequential()

model_SGD_dropout.add(Conv2D(32, (3, 3), input_shape = (64, 64, 3), activation = 'relu'))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))

model_SGD_dropout.add(Conv2D(32, (3, 3), activation = 'relu'))
model_SGD_dropout.add(MaxPooling2D(pool_size = (2, 2)))


model_SGD_dropout.add(Flatten())

# dropout in the input layer with weight constraint
model_SGD_dropout.add(Dropout(0.2, input_shape=(128,)))
model_SGD_dropout.add(Dense(units = 128, activation = 'relu', kernel_constraint=maxnorm(3)))
model_SGD_dropout.add(Dense(units = 128, activation = 'relu', kernel_constraint=maxnorm(3)))
model_SGD_dropout.add(Dense(units = 1, activation = 'sigmoid'))

# Compile model
model_SGD_dropout.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
model_SGD_dropout.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5
#accuracy_train: 0.8808  test_accuracy: 0.8110

Epoch 1/25
250/250 [==============================] - 82s 327ms/step - loss: 0.6834 - accuracy: 0.5493 - val_loss: 0.6169 - val_accuracy: 0.6400
Epoch 2/25
250/250 [==============================] - 88s 351ms/step - loss: 0.6261 - accuracy: 0.6506 - val_loss: 0.4239 - val_accuracy: 0.6950
Epoch 3/25
250/250 [==============================] - 90s 361ms/step - loss: 0.5872 - accuracy: 0.6885 - val_loss: 0.5574 - val_accuracy: 0.7170
Epoch 4/25
250/250 [==============================] - 95s 380ms/step - loss: 0.5570 - accuracy: 0.7089 - val_loss: 0.5949 - val_accuracy: 0.7255
Epoch 5/25
250/250 [==============================] - 98s 392ms/step - loss: 0.5361 - accuracy: 0.7251 - val_loss: 0.5230 - val_accuracy: 0.7520
Epoch 6/25
250/250 [==============================] - 96s 384ms/step - loss: 0.5178 - accuracy: 0.7379 - val_loss: 0.8126 - val_accuracy: 0.7525
Epoch 7/25
250/250 [==============================] - 97s 389ms/step - loss: 0.4862 - accuracy: 0.7651 - val_loss: 0.5721 - val_ac

In [21]:
from keras.layers import Dropout



model = Sequential()
#améliorer le modèle en augmentant la taille de l'image
#ajouter deux couches convolution 
model.add(Conv2D(32, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(32, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), input_shape = (150, 150, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Conv2D(64, (3, 3), activation = 'relu'))
model.add(MaxPooling2D(pool_size = (2, 2)))

model.add(Flatten())

# ajouter des couches complétement connectées et le dropout après les couches
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 128, activation = 'relu'))
model.add(Dropout(0.3))
model.add(Dense(units = 1, activation = 'sigmoid'))



In [22]:
# Compile model
model.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255, # l'echelle de chaque pixel [0-1]
                                   shear_range = 0.2, #transvection des images
                                   zoom_range = 0.2, # zoomer des images
                                   horizontal_flip = True) # rotation

test_datagen = ImageDataGenerator(rescale = 1./255)


training_set = train_datagen.flow_from_directory('dataset/training_set',
                                                 target_size = (150, 150),
                                                 batch_size = 32, #lot d'observationspour les maj
                                                 class_mode = 'binary')


test_set = test_datagen.flow_from_directory('dataset/test_set',
                                            target_size = (150, 150),
                                            batch_size = 32,
                                            class_mode = 'binary')

model.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5


model.fit_generator(training_set,
                         steps_per_epoch = 250, #8000 images/32 (lot)
                         epochs = 25,
                         validation_data = test_set,
                         validation_steps = 63) #2000 /32=62.5

#ccuracy_train: 0.9415 val_accuracy_test: 0.8740
#penser à augmenter le nombre des epochs pour améliorer encore plus le modèle

Found 8000 images belonging to 2 classes.
Found 2000 images belonging to 2 classes.
Epoch 1/25
250/250 [==============================] - 359s 1s/step - loss: 0.6901 - accuracy: 0.5296 - val_loss: 0.7119 - val_accuracy: 0.5000
Epoch 2/25
250/250 [==============================] - 363s 1s/step - loss: 0.6806 - accuracy: 0.5601 - val_loss: 0.6137 - val_accuracy: 0.5820
Epoch 3/25
250/250 [==============================] - 367s 1s/step - loss: 0.6412 - accuracy: 0.6378 - val_loss: 0.7474 - val_accuracy: 0.5945
Epoch 4/25
250/250 [==============================] - 373s 1s/step - loss: 0.5950 - accuracy: 0.6886 - val_loss: 0.5814 - val_accuracy: 0.7245
Epoch 5/25
250/250 [==============================] - 370s 1s/step - loss: 0.5686 - accuracy: 0.7180 - val_loss: 0.6550 - val_accuracy: 0.7545
Epoch 6/25
250/250 [==============================] - 371s 1s/step - loss: 0.5202 - accuracy: 0.7446 - val_loss: 0.6012 - val_accuracy: 0.7460
Epoch 7/25
250/250 [==============================] - 372s